# Mais sobre `SELECT`

- temporary tables
- views
- variables
- subqueries


In [1]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados',
        database='sakila',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

## Aquecimento

Quanta receita foi gerada para cada categoria de filmes? Liste do maior para o menor.

In [2]:
#Temos que ver quantas vezes recebi o pagamento daquele filme
# Toda vez que agrupo, agrego (SUM ou whereverrrrr)
db('''
SELECT 
    name, SUM(amount) as receita
FROM
    payment
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY
    category_id
ORDER BY
    receita DESC

''')

Executando query:
('Sports', Decimal('138295.47'))
('Animation', Decimal('137116.48'))
('Action', Decimal('130684.74'))
('Family', Decimal('129048.71'))
('Sci-Fi', Decimal('127768.37'))
('Drama', Decimal('122052.28'))
('Documentary', Decimal('122046.18'))
('Foreign', Decimal('120780.50'))
('Games', Decimal('114451.03'))
('New', Decimal('110441.67'))
('Comedy', Decimal('110384.27'))
('Children', Decimal('109727.41'))
('Classics', Decimal('108921.82'))
('Horror', Decimal('98382.66'))
('Travel', Decimal('97279.09'))
('Music', Decimal('95838.49'))


Liste os filmes e o numero de vezes em que foram alugados

In [3]:
#Depois de fazer essa filtragem vou ter uma tabela com os filmes repetidos e o seu id. Aí tenho que agrupar pelo mesmo nome
# e depois que agrupo, agrego com o COUNT.

db('''
SELECT 
    title, COUNT(rental_id) as vezes_alugados
FROM
    film
    INNER JOIN inventory USING (film_id)
    INNER JOIN rental USING (inventory_id)
GROUP BY
    film_id
ORDER BY
    vezes_alugados DESC
''')

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('FORWARD TEMPLE', 32)
('GRIT CLOCKWORK', 32)
('JUGGLER HARDLY', 32)
('RIDGEMONT SUBMARINE', 32)
('SCALAWAG DUCK', 32)
('APACHE DIVINE', 31)
('GOODFELLAS SALUTE', 31)
('HOBBIT ALIEN', 31)
('NETWORK PEAK', 31)
('ROBBERS JOON', 31)
('RUSH GOODFELLAS', 31)
('TIMBERLAND SKY', 31)
('WIFE TURN', 31)
('ZORRO ARK', 31)
('BUTTERFLY CHOCOLAT', 30)
('CAT CONEHEADS', 30)
('DOGMA FAMILY', 30)
('ENGLISH BULWORTH', 30)
('FROST HEAD', 30)
('GRAFFITI LOVE', 30)
('HARRY IDAHO', 30)
('IDOLS SNATCHERS', 30)
('MARRIED GO', 30)
('MASSACRE USUAL', 30)
('MUSCLE BRIGHT', 30)
('PULP BEVERLY', 30)
('RUGRATS SHAKESPEARE', 30)
('SHOCK CABIN', 30)
('SUSPECTS QUILLS', 30)
('WITCHES PANIC', 30)
('BINGO TALENTED', 29)
('BOOGIE AMELIE', 29)
('CONFIDENTIAL INTERVIEW', 29)
('DEER VIRGINIAN', 29)
('ENEMY ODDS', 29)
('FAMILY SWEET', 29)
('GLEAMING JAWBREAKER', 29)
('GREATEST NORTH', 29)
('MOON BUNCH', 29)
('STORM HAPPINESS', 29)
('SWEETHEARTS SUSPECTS', 

Liste os filmes e o numero de vezes em que foram alugados apenas para filmes que foram alugados mais do que a média de numero de alugueis por filme.

In [4]:
db('''
SELECT 
    COUNT(film_id) INTO @cnt_film
FROM
    film
''')

db('''
SELECT
    COUNT(rental_id) INTO @cnt_rental
FROM
    rental
''')

db('''
SELECT 
    @cnt_rental/@cnt_film INTO @avg_rental_per_film
''')

db('''
SELECT
    @avg_rental_per_film

''')

Executando query:
Executando query:
Executando query:
Executando query:
(Decimal('16.044000000'),)


In [5]:
#Depois de fazer essa filtragem vou ter uma tabela com os filmes repetidos e o seu id. Aí tenho que agrupar pelo mesmo nome
# e depois que agrupo, agrego com o COUNT.

db('''
SELECT 
    title, COUNT(rental_id) as vezes_alugados
FROM
    film
    INNER JOIN inventory USING (film_id)
    INNER JOIN rental USING (inventory_id)
GROUP BY
    film_id
HAVING
    vezes_alugados > @avg_rental_per_film
ORDER BY
    vezes_alugados ASC
''')

Executando query:
('BEACH HEARTBREAKERS', 17)
('BIKINI BORROWERS', 17)
('BREAKFAST GOLDFINGER', 17)
('COMFORTS RUSH', 17)
('CONFUSED CANDLES', 17)
('CONVERSATION DOWNHILL', 17)
('DYING MAKER', 17)
('EGYPT TENENBAUMS', 17)
('FACTORY DRAGON', 17)
('FERRIS MOTHER', 17)
('FLINTSTONES HAPPINESS', 17)
('FREAKY POCUS', 17)
('FRENCH HOLIDAY', 17)
('GANDHI KWAI', 17)
('GREEK EVERYONE', 17)
('HANOVER GALAXY', 17)
('HOLOCAUST HIGHBALL', 17)
('IDAHO LOVE', 17)
('INTENTIONS EMPIRE', 17)
('JEKYLL FROGMEN', 17)
('JERICHO MULAN', 17)
('JOON NORTHWEST', 17)
('LADY STAGE', 17)
('LAWLESS VISION', 17)
('LEAGUE HELLFIGHTERS', 17)
('LOUISIANA HARRY', 17)
('MAJESTIC FLOATS', 17)
('MASK PEACH', 17)
('MIDNIGHT WESTWARD', 17)
('MUSKETEERS WAIT', 17)
('NEWTON LABYRINTH', 17)
('NOON PAPI', 17)
('ORIENT CLOSER', 17)
('PERFECT GROOVE', 17)
('POSEIDON FOREVER', 17)
('POTTER CONNECTICUT', 17)
('REAR TRADING', 17)
('ROCK INSTINCT', 17)
('ROOTS REMEMBER', 17)
('SATISFACTION CONFIDENTIAL', 17)
('SLACKER LIAISONS', 17)
(

## Views

Uma *view* é uma tabela virtual, construida a partir de um comando `SELECT`. Por exemplo: execute o código a seguir.

In [6]:
#Quantas vezes foram alugados mesmo para filmes que nunca foram alugados: com cnt igual a 0
#Todos os filmes serão presentes mesmo que não haja [cópia física em lugar nenhum = inventory]
# e mesmo que nunca tenha sido alugado na vida
#pode ter quantos nulos for, ele conta como 0
db('''
DROP VIEW IF EXISTS movie_count;
''')

db('''
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
''')

Executando query:
Executando query:


Agora temos uma *view* chamada `movie_count`. Vamos verificar que ela funcionou, listando as 30 primeiras linhas:

In [7]:
db('''
SELECT 
    *
FROM
    movie_count 
LIMIT 
    30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


In [8]:
db('''
SELECT 
    * 
FROM 
    movie_count 
ORDER BY
    cnt DESC
LIMIT 30
''')

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('JUGGLER HARDLY', 32)
('GRIT CLOCKWORK', 32)
('FORWARD TEMPLE', 32)
('SCALAWAG DUCK', 32)
('RIDGEMONT SUBMARINE', 32)
('ZORRO ARK', 31)
('WIFE TURN', 31)
('TIMBERLAND SKY', 31)
('RUSH GOODFELLAS', 31)
('ROBBERS JOON', 31)
('NETWORK PEAK', 31)
('HOBBIT ALIEN', 31)
('GOODFELLAS SALUTE', 31)
('APACHE DIVINE', 31)
('WITCHES PANIC', 30)
('SUSPECTS QUILLS', 30)
('SHOCK CABIN', 30)
('RUGRATS SHAKESPEARE', 30)
('PULP BEVERLY', 30)
('MUSCLE BRIGHT', 30)
('MASSACRE USUAL', 30)
('MARRIED GO', 30)
('IDOLS SNATCHERS', 30)
('HARRY IDAHO', 30)
('GRAFFITI LOVE', 30)
('FROST HEAD', 30)
('ENGLISH BULWORTH', 30)
('DOGMA FAMILY', 30)


Agora suponha que alteramos a tabela `film`, mudando o nome do filme "DAZED PUNK" para "STONED PUNK".

**Atividade**: Do it.

In [9]:
db('''
UPDATE film SET title="STONED PUNK" WHERE title="DAZED PUNK"
''')

Executando query:


Verifique agora a nossa *view*:

In [10]:
db('''
SELECT
    *
FROM
    movie_count
LIMIT
    30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


Como você pode ver, as views são tabelas virtuais que são automaticamente atualizadas quando as tabelas originais são modificadas.

In [11]:
connection.rollback()

In [12]:
db('''
SELECT
    *
FROM
    movie_count
LIMIT
    30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


In [13]:
db('''
UPDATE film SET title="DAZED PUNK" WHERE title="STONED PUNK"
''')

Executando query:


In [14]:
db('''
SELECT
    *
FROM
    movie_count
LIMIT
    30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('DAZED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


### Vamos praticar

Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view `movie_count`

In [15]:
db('''
SELECT
    title, cnt
FROM 
    movie_count
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:
('COWBOY DOOM', 7)


Registre um aluguel do filme "COWBOY DOOM", feito pelo funcionario "Jon Stephens" na loja id=1 para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'

In [16]:
db('''
SELECT 
    film_id INTO @film_id  
FROM
    film
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:


In [17]:
db('''
SELECT
    *
FROM
    film
    INNER JOIN inventory USING (film_id)
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:
(185, 'COWBOY DOOM', 'A Astounding Drama of a Boy And a Lumberjack who must Fight a Butler in A Baloon', 2006, 1, None, 3, Decimal('2.99'), 146, Decimal('10.99'), 'PG', {'Commentaries', 'Deleted Scenes', 'Behind the Scenes'}, datetime.datetime(2006, 2, 15, 5, 3, 42), 845, 1, datetime.datetime(2006, 2, 15, 5, 9, 17))
(185, 'COWBOY DOOM', 'A Astounding Drama of a Boy And a Lumberjack who must Fight a Butler in A Baloon', 2006, 1, None, 3, Decimal('2.99'), 146, Decimal('10.99'), 'PG', {'Commentaries', 'Deleted Scenes', 'Behind the Scenes'}, datetime.datetime(2006, 2, 15, 5, 3, 42), 846, 1, datetime.datetime(2006, 2, 15, 5, 9, 17))


In [18]:
db('''
SELECT 
    staff_id INTO @staff_id  
FROM
    staff
WHERE
    first_name = 'Jon'
    AND last_name = 'Stephens'
''')

Executando query:


In [19]:
db('''
SELECT 
    customer_id INTO @customer_id  
FROM
    customer
WHERE
    first_name = 'JESSIE'
    AND last_name = 'BANKS'
''')

Executando query:


In [20]:
db('''
SELECT
    MIN(inventory_id)
FROM
    film
    INNER JOIN inventory USING (film_id)
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:
(845,)


In [21]:
db('''
INSERT INTO rental (
    rental_date, 
    inventory_id, 
    customer_id, 
    return_date, 
    staff_id
)
VALUES 
(
    "2019-01-01",
    @inventory_id,
    @customer_id,
    "2019-01-08",
    @staff_id
)''')

Executando query:


IntegrityError: 1048 (23000): Column 'inventory_id' cannot be null

Verifique usando a view que a contagem de alugueis do filme subiu.

In [22]:
db('''
SELECT
    title, cnt
FROM 
    movie_count
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:
('COWBOY DOOM', 7)


Faça o *rollback* desta alteração, para não modificar a nossa querida base de dados *sakila*.

In [23]:
connection.rollback()

In [24]:
db('''
SELECT
    title, cnt
FROM 
    movie_count
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:
('COWBOY DOOM', 7)


## Tabelas temporárias

Tabelas temporárias podem ser criadas para ajudar nas tarefas de manipulação de dados. Essas tabelas existem apenas pela duração da sessão. Para criar uma tabela temporária, basta adicionar a palavra-chave `TEMPORARY` no momento da criação.

É comum criar tabelas temporárias à partir do resultado de comandos `SELECT`. Por exemplo, a seguinte query permite montar uma tabela temporária com os filmes que duram mais que 3 horas:

In [25]:
db('''
DROP TABLE IF EXISTS long_film

''')

db('''
CREATE TEMPORARY TABLE long_film 
    SELECT
        *
    FROM
        film
    WHERE
        film.length > 180;
''')

Executando query:
Executando query:


Podemos verificar que a tabela `long_film` agora existe:

In [26]:
db('DESCRIBE long_film')

Executando query:
('film_id', 'smallint unsigned', 'NO', '', '0', 'NULL')
('title', 'varchar(128)', 'NO', '', None, 'NULL')
('description', 'text', 'YES', '', None, 'NULL')
('release_year', 'year', 'YES', '', None, 'NULL')
('language_id', 'tinyint unsigned', 'NO', '', None, 'NULL')
('original_language_id', 'tinyint unsigned', 'YES', '', None, 'NULL')
('rental_duration', 'tinyint unsigned', 'NO', '', '3', 'NULL')
('rental_rate', 'decimal(4,2)', 'NO', '', '4.99', 'NULL')
('length', 'smallint unsigned', 'YES', '', None, 'NULL')
('replacement_cost', 'decimal(5,2)', 'NO', '', '19.99', 'NULL')
('rating', "enum('G','PG','PG-13','R','NC-17')", 'YES', '', 'G', 'NULL')
('special_features', "set('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')", 'YES', '', None, 'NULL')
('last_update', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', 'on update CURRENT_TIMESTAMP')


Muito embora ela não apareça na lista de tabelas: isso é um bug do MySQL. (https://dev.mysql.com/worklog/task/?id=648)

In [27]:
db('SHOW TABLES')

Executando query:
('actor',)
('actor_info',)
('address',)
('category',)
('city',)
('country',)
('customer',)
('customer_list',)
('film',)
('film_actor',)
('film_category',)
('film_list',)
('film_text',)
('inventory',)
('language',)
('movie_count',)
('nicer_but_slower_film_list',)
('payment',)
('rental',)
('sales_by_film_category',)
('sales_by_store',)
('staff',)
('staff_list',)
('store',)


Vamos listar o conteudo desta tabela:

In [28]:
db('SELECT title FROM long_film')

Executando query:
('ANALYZE HOOSIERS',)
('BAKED CLEOPATRA',)
('CATCH AMISTAD',)
('CHICAGO NORTH',)
('CONSPIRACY SPIRIT',)
('CONTROL ANTHEM',)
('CRYSTAL BREAKING',)
('DARN FORRESTER',)
('FRONTIER CABIN',)
('GANGS PRIDE',)
('HAUNTING PIANIST',)
('HOME PITY',)
('HOTEL HAPPINESS',)
('INTRIGUE WORST',)
('JACKET FRISCO',)
('KING EVOLUTION',)
('LAWLESS VISION',)
('LOVE SUICIDES',)
('MONSOON CAUSE',)
('MOONWALKER FOOL',)
('MUSCLE BRIGHT',)
('POND SEATTLE',)
('RECORDS ZORRO',)
('REDS POCUS',)
('RUNAWAY TENENBAUMS',)
('SATURN NAME',)
('SCALAWAG DUCK',)
('SEARCHERS WAIT',)
('SMOOCHY CONTROL',)
('SOLDIERS EVOLUTION',)
('SONS INTERVIEW',)
('SORORITY QUEEN',)
('STAR OPERATION',)
('SWEET BROTHERHOOD',)
('THEORY MERMAID',)
('WIFE TURN',)
('WILD APOLLO',)
('WORST BANGER',)
('YOUNG LANGUAGE',)


Vamos apagar a tabela `long_film`:

In [29]:
db('DROP TABLE long_film')

Executando query:


### Vamos praticar

- Crie uma tabela temporária `max_duration` que contém a duração máxima de filme para cada categoria

In [30]:
db('''
DROP TABLE IF EXISTS max_duration

''')

db('''
CREATE TEMPORARY TABLE max_duration 
    SELECT
        category_id, name, MAX(length) as duracaoMax
    FROM
        film
        INNER JOIN film_category USING (film_id)
        INNER JOIN category USING (category_id)
    GROUP BY
        category_id
    ORDER BY
        duracaoMax DESC
    
''')

Executando query:
Executando query:


 - Verifique a tabela.

In [31]:
db('''
SELECT
    *
FROM 
    max_duration
''')

Executando query:
(1, 'Action', 185)
(2, 'Animation', 185)
(5, 'Comedy', 185)
(10, 'Games', 185)
(12, 'Music', 185)
(14, 'Sci-Fi', 185)
(16, 'Travel', 185)
(4, 'Classics', 184)
(8, 'Family', 184)
(9, 'Foreign', 184)
(15, 'Sports', 184)
(6, 'Documentary', 183)
(13, 'New', 183)
(7, 'Drama', 181)
(11, 'Horror', 181)
(3, 'Children', 178)


In [32]:
db('DESCRIBE max_duration')

Executando query:
('category_id', 'tinyint unsigned', 'NO', '', None, 'NULL')
('name', 'varchar(25)', 'NO', '', None, 'NULL')
('duracaoMax', 'smallint unsigned', 'YES', '', None, 'NULL')


- Agora use a tabela temporária para construir uma tabela com as categorias e seus respectivos filmes mais longos:

In [33]:
db('''
SELECT
    length, duracaoMax, title, name
FROM
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN max_duration USING (category_id)
WHERE
    length = duracaoMax
''')

Executando query:
(185, 185, 'DARN FORRESTER', 'Action')
(185, 185, 'WORST BANGER', 'Action')
(185, 185, 'GANGS PRIDE', 'Animation')
(185, 185, 'POND SEATTLE', 'Animation')
(185, 185, 'CONTROL ANTHEM', 'Comedy')
(185, 185, 'CHICAGO NORTH', 'Games')
(185, 185, 'HOME PITY', 'Music')
(185, 185, 'SOLDIERS EVOLUTION', 'Sci-Fi')
(185, 185, 'MUSCLE BRIGHT', 'Travel')
(185, 185, 'SWEET BROTHERHOOD', 'Travel')
(184, 184, 'CONSPIRACY SPIRIT', 'Classics')
(184, 184, 'KING EVOLUTION', 'Family')
(184, 184, 'CRYSTAL BREAKING', 'Foreign')
(184, 184, 'SORORITY QUEEN', 'Foreign')
(184, 184, 'SMOOCHY CONTROL', 'Sports')
(183, 183, 'WIFE TURN', 'Documentary')
(183, 183, 'YOUNG LANGUAGE', 'Documentary')
(183, 183, 'FRONTIER CABIN', 'New')
(181, 181, 'JACKET FRISCO', 'Drama')
(181, 181, 'ANALYZE HOOSIERS', 'Horror')
(181, 181, 'LOVE SUICIDES', 'Horror')
(178, 178, 'FURY MURDER', 'Children')
(178, 178, 'WRONG BEHAVIOR', 'Children')


- delete a tabela temporária

In [34]:
db('DROP TABLE max_duration')

Executando query:


### Desafio!

- Gere uma tabela contendo, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [35]:
#Penelope Guiness Animation, ...

db('DROP TABLE IF EXISTS actor_category_films')

db('''
CREATE TEMPORARY TABLE actor_category_films
    SELECT 
        first_name, last_name, CONCAT(name, ":", GROUP_CONCAT(title SEPARATOR ",")) as films

    FROM
        actor
        INNER JOIN film_actor USING (actor_id)
        INNER JOIN film USING (film_id)
        INNER JOIN film_category USING (film_id)
        INNER JOIN category USING (category_id)
    GROUP BY
        actor_id, category_id

''')

Executando query:
Executando query:


In [36]:
db('SELECT * FROM actor_category_films LIMIT 10')

Executando query:
('PENELOPE', 'GUINESS', 'Animation:ANACONDA CONFESSIONS')
('PENELOPE', 'GUINESS', 'Children:LANGUAGE COWBOY')
('PENELOPE', 'GUINESS', 'Classics:COLOR PHILADELPHIA,WESTWARD SEABISCUIT')
('PENELOPE', 'GUINESS', 'Comedy:VERTIGO NORTHWEST')
('PENELOPE', 'GUINESS', 'Documentary:ACADEMY DINOSAUR')
('PENELOPE', 'GUINESS', 'Family:KING EVOLUTION,SPLASH GUMP')
('PENELOPE', 'GUINESS', 'Foreign:MULHOLLAND BEAST')
('PENELOPE', 'GUINESS', 'Games:BULWORTH COMMANDMENTS,HUMAN GRAFFITI')
('PENELOPE', 'GUINESS', 'Horror:ELEPHANT TROJAN,LADY STAGE,RULES HUMAN')
('PENELOPE', 'GUINESS', 'Music:WIZARD COLDBLOODED')


In [37]:
db('''
SELECT 
    MIN(first_name), MIN(last_name), GROUP_CONCAT(films SEPARATOR ";") as total_films
FROM
    actor_category_films
GROUP BY
    actor_id
''')

Executando query:


ProgrammingError: 1054 (42S22): Unknown column 'actor_id' in 'group statement'

## Variáveis

Podemos montar uma query que retorne um valor só e armazenar este valor em uma variável, para uso posterior em outras queries. Para isso vamos usar o prefixo '@' para indicar variáveis, e o comando `SELECT ... INTO`.

Exemplo: quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

Primeiro vamos calcular a média e o desvio padrão dos preços de aluguel:

In [38]:
db('''
SELECT 
    AVG(rental_rate), 
    STDDEV(rental_rate)
INTO 
    @avg_rate, 
    @stddev_rate 
FROM
    film;
''')

Executando query:


Note que a query não retorna um resultado: o resultado foi armazenado direto nas variáveis `@avg_rate` e `@stddev_rate`. Vamos usar um `SELECT` sem tabelas para ver o resultado:

In [39]:
db('SELECT @avg_rate, @stddev_rate')

Executando query:
(Decimal('2.980000000'), 1.6455698101265719)


Agora podemos selecionar os filmes caros!

In [40]:
db('''
SELECT 
    title, rental_rate
FROM
    film
WHERE
    rental_rate > @avg_rate + @stddev_rate;
''')

Executando query:
('ACE GOLDFINGER', Decimal('4.99'))
('AIRPLANE SIERRA', Decimal('4.99'))
('AIRPORT POLLOCK', Decimal('4.99'))
('ALADDIN CALENDAR', Decimal('4.99'))
('ALI FOREVER', Decimal('4.99'))
('AMELIE HELLFIGHTERS', Decimal('4.99'))
('AMERICAN CIRCUS', Decimal('4.99'))
('ANTHEM LUKE', Decimal('4.99'))
('APACHE DIVINE', Decimal('4.99'))
('APOCALYPSE FLAMINGOS', Decimal('4.99'))
('ATTACKS HATE', Decimal('4.99'))
('ATTRACTION NEWTON', Decimal('4.99'))
('AUTUMN CROW', Decimal('4.99'))
('BABY HALL', Decimal('4.99'))
('BACKLASH UNDEFEATED', Decimal('4.99'))
('BEAST HUNCHBACK', Decimal('4.99'))
('BEAUTY GREASE', Decimal('4.99'))
('BEHAVIOR RUNAWAY', Decimal('4.99'))
('BETRAYED REAR', Decimal('4.99'))
('BIKINI BORROWERS', Decimal('4.99'))
('BILKO ANONYMOUS', Decimal('4.99'))
('BIRCH ANTITRUST', Decimal('4.99'))
('BIRD INDEPENDENCE', Decimal('4.99'))
('BIRDS PERDITION', Decimal('4.99'))
('BLINDNESS GUN', Decimal('4.99'))
('BOILED DARES', Decimal('4.99'))
('BOOGIE AMELIE', Decimal('4.99')

### Vamos praticar

Use variáveis temporárias para encontrar o ator que mais participou de filmes.

## Operador `IN`

Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [41]:
db('''
SELECT 
    actor_id, first_name, last_name, COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    num_films DESC
LIMIT 3
''')

Executando query:
(107, 'GINA', 'DEGENERES', 42)
(102, 'WALTER', 'TORN', 41)
(198, 'MARY', 'KEITEL', 40)


Vamos criar uma tabela temporária para guardar a informação de `actor_id` desses atores:

In [42]:
db('DROP TABLE IF EXISTS temp_pop_actors')
db('''
CREATE TEMPORARY TABLE temp_pop_actors
    SELECT first_name, last_name, actor_id FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    GROUP BY 
        actor_id
    ORDER BY 
        COUNT(film_id) DESC
    LIMIT 3
''')
db('''
SELECT * from temp_pop_actors
''')

Executando query:
Executando query:
Executando query:
('GINA', 'DEGENERES', 107)
('WALTER', 'TORN', 102)
('MARY', 'KEITEL', 198)


Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [43]:
db('''
SELECT DISTINCT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
''')

Executando query:
('BED HIGHBALL',)
('CALENDAR GUNFIGHT',)
('CHAMBER ITALIAN',)
('CHAPLIN LICENSE',)
('CHARIOTS CONSPIRACY',)
('CLUELESS BUCKET',)
('COLDBLOODED DARLING',)
('CONEHEADS SMOOCHY',)
('DARKNESS WAR',)
('DEER VIRGINIAN',)
('DOGMA FAMILY',)
('ELEPHANT TROJAN',)
('EXCITEMENT EVE',)
('FRISCO FORREST',)
('GANDHI KWAI',)
('GOODFELLAS SALUTE',)
('GUNFIGHT MOON',)
('HALL CASSIDY',)
('HEARTBREAKERS BRIGHT',)
('HOOK CHARIOTS',)
('HYDE DOCTOR',)
('IMPACT ALADDIN',)
('INDIAN LOVE',)
('INTRIGUE WORST',)
('LICENSE WEEKEND',)
('LOUISIANA HARRY',)
('MAGNIFICENT CHITTY',)
('METAL ARMAGEDDON',)
('MIDNIGHT WESTWARD',)
('MOVIE SHAKESPEARE',)
('MUMMY CREATURES',)
('OPEN AFRICAN',)
('SEARCHERS WAIT',)
('SEVEN SWARM',)
('SIERRA DIVIDE',)
('SPIRITED CASUALTIES',)
('STORM HAPPINESS',)
('SUGAR WONKA',)
('TELEGRAPH VOYAGE',)
('TRAINSPOTTING STRANGERS',)
('WIFE TURN',)
('WINDOW SIDE',)
('AMELIE HELLFIGHTERS',)
('ARABIA DOGMA',)
('BANG KWAI',)
('CASABLANCA SUPER',)
('CASPER DRAGONFLY',)
('CROW GREASE',

Note o uso de *subqueries*!

Não se esqueça de limpar tudo no final!

In [44]:
db('DROP TABLE temp_pop_actors')

Executando query:


### Vamos praticar

Liste os atores que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora).

In [53]:
db('DROP TABLE IF EXISTS filmes_rentaveis')


db('''
CREATE TEMPORARY TABLE filmes_rentaveis
    SELECT 
        film_id, SUM(amount) AS totalPreco
    FROM
        payment
        INNER JOIN rental USING (rental_id)
        INNER JOIN inventory USING (inventory_id)
    GROUP BY 
        film_id
    ORDER BY 
        totalPreco DESC
    LIMIT 3
''')

db(''' SELECT * FROM filmes_rentaveis''')

Executando query:
Executando query:
Executando query:
(879, Decimal('231.73'))
(973, Decimal('223.69'))
(1000, Decimal('214.69'))


In [57]:
db('''
SELECT DISTINCT
    first_name, last_name
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
WHERE
    film_id IN (SELECT film_id FROM filmes_rentaveis);

''')

Executando query:
('WOODY', 'HOFFMAN')
('CARMEN', 'HUNT')
('GINA', 'DEGENERES')
('LUCILLE', 'DEE')
('VIVIEN', 'BASINGER')
('MICHAEL', 'BENING')
('THORA', 'TEMPLE')
('JULIA', 'BARRYMORE')
('CAMERON', 'ZELLWEGER')
('GRETA', 'MALDEN')
('MICHAEL', 'BOLGER')
('IAN', 'TANDY')
('NICK', 'DEGENERES')
('LISA', 'MONROE')


## Subqueries

Os tópicos discutidos acima poderiam ser resolvidos, em grande parte, com subqueries. As subqueries são queries `SELECT` criadas dentro de outras queries. 

Poderíamos ter usado subqueries nos mesmos lugares onde usamos tabelas temporárias, nos tópicos acima. Quando a subquery pode ser transformada em uma tabela temporária independente, separada da query exterior, dizemos que a subquery é **não-correlacionada** com a query exterior.

Usar subqueries não-correlacionadas é um tópico controverso: podemos sempre usar uma tabela temporária ou, ás vezes, pensar em um `JOIN` simples. Aliás, muitas vezes o otimizador de queries do banco de dados transformará a subquery em `JOIN`, se isso for vantajoso em termos de desempenho.

Uma subquery que depende da query externa (e portanto não pode ser separada em uma tabela temporária independente) é chamada de **subquery correlacionada**. Nestes casos podemos ter que executar a subquery para cada linha da query exterior! 

### Vamos praticar

Vamos refazer a atividade dos filmes de atores populares, usando subqueries. 

Temos um problema: o MySQL não suporta ``LIMIT`` em subqueries com o operador ``IN``. Vamos investigar isso mais de perto. 

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

In [58]:
try:
    db('''
    SELECT 
        actor_id, first_name, last_name
    FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    WHERE
        film_actor.film_id IN (
            SELECT 
                film_id
            FROM
                film
                INNER JOIN inventory USING (film_id)
                INNER JOIN rental USING (inventory_id)
                INNER JOIN payment USING (rental_id)
            WHERE
                payment_date IS NOT NULL
            GROUP BY 
                film_id
            ORDER BY 
                SUM(amount)
            LIMIT 
                3)
    ORDER BY 
        actor_id
    ''')
except mysql.connector.ProgrammingError as e:
    print(f'ProgrammingError: {e}')

Executando query:
ProgrammingError: 1235 (42000): This version of MySQL doesn't yet support 'LIMIT & IN/ALL/ANY/SOME subquery'


Ok, apareceu o problema. Mas considere que o problema original não precisava de IN desde o começo! Construa essa solução.

# `UNION`

Quando duas tabelas tem **EXATAMENTE** as mesmas colunas, podemos concatená-las e formar uma grande tabela unificada usando o operador `UNION`. Por exemplo: suponha que desejamos montar uma lista dos nomes e sobrenomes de todos os clientes E de todos os funcionários. Eis uma solução possível:

In [59]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('''
CREATE TEMPORARY TABLE nomes_clientes 
    SELECT first_name, last_name FROM customer
''')

Executando query:
Executando query:


In [60]:
db('DESCRIBE nomes_clientes')
db('SELECT * FROM nomes_clientes LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')
('BARBARA', 'JONES')
('ELIZABETH', 'BROWN')


In [61]:
db('DROP TABLE IF EXISTS nomes_staff')
db('''
CREATE TEMPORARY TABLE nomes_staff 
    SELECT first_name, last_name FROM staff
''')

Executando query:
Executando query:


In [62]:
db('DESCRIBE nomes_staff')
db('SELECT * FROM nomes_staff LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')


In [63]:
db('DROP TABLE IF EXISTS nomes_all')
db('''
CREATE TEMPORARY TABLE nomes_all 
    (SELECT * FROM nomes_staff) 
    UNION 
    (SELECT * FROM nomes_clientes)
''')

Executando query:
Executando query:


In [64]:
db('DESCRIBE nomes_all')
db('SELECT * FROM nomes_all LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', '', 'NULL')
('last_name', 'varchar(45)', 'NO', '', '', 'NULL')
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')


In [ ]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('DROP TABLE IF EXISTS nomes_staff')
db('DROP TABLE IF EXISTS nomes_all')

**Vamos praticar:** refaça o exemplo acima mas use *subqueries* ao invés de *temp tables*.

## Desafios!

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme!

In [66]:
db('''
SELECT 
    first_name, last_name, film_id, title
FROM
    actor 
    INNER JOIN film_actor USING (actor_id)
    INNER JOIN film USING (film_id)
GROUP BY
    film_id
''')

Executando query:
('PENELOPE', 'GUINESS', 1, 'ACADEMY DINOSAUR')
('PENELOPE', 'GUINESS', 23, 'ANACONDA CONFESSIONS')
('PENELOPE', 'GUINESS', 25, 'ANGELS LIFE')
('PENELOPE', 'GUINESS', 106, 'BULWORTH COMMANDMENTS')
('PENELOPE', 'GUINESS', 140, 'CHEAPER CLYDE')
('PENELOPE', 'GUINESS', 166, 'COLOR PHILADELPHIA')
('PENELOPE', 'GUINESS', 277, 'ELEPHANT TROJAN')
('PENELOPE', 'GUINESS', 361, 'GLEAMING JAWBREAKER')
('PENELOPE', 'GUINESS', 438, 'HUMAN GRAFFITI')
('PENELOPE', 'GUINESS', 499, 'KING EVOLUTION')
('PENELOPE', 'GUINESS', 506, 'LADY STAGE')
('PENELOPE', 'GUINESS', 509, 'LANGUAGE COWBOY')
('PENELOPE', 'GUINESS', 605, 'MULHOLLAND BEAST')
('PENELOPE', 'GUINESS', 635, 'OKLAHOMA JUMANJI')
('PENELOPE', 'GUINESS', 749, 'RULES HUMAN')
('PENELOPE', 'GUINESS', 832, 'SPLASH GUMP')
('PENELOPE', 'GUINESS', 939, 'VERTIGO NORTHWEST')
('PENELOPE', 'GUINESS', 970, 'WESTWARD SEABISCUIT')
('PENELOPE', 'GUINESS', 980, 'WIZARD COLDBLOODED')
('NICK', 'WAHLBERG', 3, 'ADAPTATION HOLES')
('NICK', 'WAHLBERG', 

In [75]:
db('''
SELECT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    LEFT(title, 1) = LEFT(first_name, 1)
GROUP BY
    film_id
HAVING
    COUNT(actor_id) > 2
''')

Executando query:
('CROW GREASE',)
('JEDI BENEATH',)
('SUBMARINE BED',)


Semana do "DAN HARRIS": liste os clientes que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas onde a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'

In [78]:
db('DROP TABLE IF EXISTS films_dan')
db('''
CREATE TEMPORARY TABLE films_dan
    SELECT DISTINCT
        film_id
    FROM
        film_actor 
        INNER JOIN actor USING (actor_id)
    WHERE
        first_name = "DAN" AND last_name = "HARRIS"
    ORDER BY
        film_id
''')
db('SELECT * FROM films_dan')

Executando query:
Executando query:
Executando query:
(63,)
(87,)
(226,)
(236,)
(298,)
(307,)
(354,)
(383,)
(417,)
(421,)
(457,)
(462,)
(474,)
(521,)
(593,)
(728,)
(750,)
(769,)
(781,)
(795,)
(844,)
(851,)
(862,)
(868,)
(892,)
(893,)
(936,)
(965,)


In [79]:
db('''
SELECT
    COUNT(DISTINCT customer_id)
FROM
    customer
    LEFT OUTER JOIN rental USING (customer_id)
    LEFT OUTER JOIN inventory USING (inventory_id)
    LEFT OUTER JOIN films_dan USING (film_id)
WHERE
    films_dan.film_id IS NULL
''')

Executando query:
(599,)


## Conclusão

Façamos uma pausa para apreciar quão longe estamos: já conseguimos criar nossas tabelas, inserir informação, removê-la, atualizá-la, e consultar nossa base de maneiras bem sofisticadas! Vimos desde `SELECT` simples até buscas mais complexas envolvendo várias etapas de processamento para obter o dado desejado.

Por hoje é só, feche sua conexão e bom descanso!

In [80]:
connection.close()